In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_COPA")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
CONTEXT_COL = "premise"
QUESTION_COL = "question"
CHOICE_1_COL = "choice1"
CHOICE_2_COL = "choice2"


def preprocess_function(examples):
    """
    The preprocessing function needs to:
    1. Make two copies of the CONTEXT_COL field and combine each of them with QUESTION_COL to recreate how a sentence starts.
    2. Combine QUESTION_COL with each of the two possible choices.
    3. Flatten these two lists so you can tokenize them, and then unflatten them afterward so each example has a corresponding input_ids, attention_mask, and labels field.
    """

    question_headers = examples[QUESTION_COL]

    # Repeat each premise two times to go with the two choice possibilities.
    first_sentences = [[context] * 2 for context in examples[CONTEXT_COL]]
    # Grab all choices possible for each context.
    second_sentences = [
        [f"{header} {examples[choice][i]}" for choice in [CHOICE_1_COL, CHOICE_2_COL]] for i, header
        in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    return {k: [v[i: i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'question', 'choice1', 'choice2'], batched=True)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
import numpy as np


def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
# from transformers import set_seed
#
# set_seed(42)

model = AutoModelForMultipleChoice.from_pretrained(model_name)
model.config.use_cache = False

In [ ]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [ ]:
loha_model = get_peft_model(model, peft_config)
loha_model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_copa"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=loha_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub(new_model_name)

In [ ]:
# Example
choice1 = "V soseski je zmanjkalo elektrike."
choice2 = "Napovedan je bil močan veter."
prompt = "Na daljnovod je padlo drevo."
question = "effect"

In [ ]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
# set_seed(42)

adapter_name = "anzeo/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
inputs = tokenizer([[prompt, f"{question} {choice1}"], [prompt, f"{question} {choice2}"]], return_tensors="pt",
                   padding=True)
labels = torch.tensor(0).unsqueeze(0)

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained(adapter_name)
outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
logits = outputs.logits

In [ ]:
# Print prediction
logits.argmax().item()